# Leitura inicial - dados IBGE
## objetivo: 
Padronizar, limpar e organizar os dados de bairros de Belém-PA a partir de múltiplos arquivos CSV

### Resumo Descritivo do Código

O código realiza a leitura de múltiplos arquivos CSV contendo dados de bairros de Belém-PA, tanto do conjunto `tabela3033` quanto do `tabela3170`. Após a leitura:

#### 1. Limpeza dos nomes de bairros
- Remove o sufixo `- Belém (PA)` das colunas `Bairro` em todos os DataFrames, garantindo uniformidade.

#### 2. Verificação de consistência
- Compara as listas de bairros entre diferentes DataFrames para identificar discrepâncias ou bairros ausentes.

#### 3. Salvamento seguro dos arquivos
- Cria uma função `salvar_csv` que grava os DataFrames em arquivos CSV no diretório de processamento.
- Evita sobrescrever arquivos já existentes e informa o status do salvamento.

**Resumo geral:**  
O código prepara e padroniza os dados de bairros para posterior análise, garantindo que os nomes estejam consistentes e os arquivos processados sejam salvos de forma organizada.



In [103]:
import json
import pandas as pd
import os

## 1. Open files

In [83]:
path="/home/akel/PycharmProjects/Data_ambiental/data/bairros/raw"

filename_a='tabela3033_a.csv'
filename_b='tabela3033_b.csv'
filename_c='tabela3033_c.csv'

dfa = pd.read_csv(path+filename_a,sep=';',skiprows=4)
dfb = pd.read_csv(path+filename_b,sep=';',skiprows=4)
dfc = pd.read_csv(path+filename_c,sep=';',skiprows=4)

filename_A='tabela3170_a.csv'
filename_B='tabela3170_b.csv'
filename_C='tabela3170_c.csv'
filename_D='tabela3170_d.csv'

dfA = pd.read_csv(path+filename_A,sep=';',skiprows=5)
dfB = pd.read_csv(path+filename_B,sep=';',skiprows=5)
dfC = pd.read_csv(path+filename_C,sep=';',skiprows=5)
dfD = pd.read_csv(path+filename_D,sep=';',skiprows=5)
#dfe = pd.read_csv(path+filename_e,sep=';',skiprows=4)


## 2. Removendo sufixo

In [84]:
#removendo o sufixo da variavel bairro  - Belém (PA)
dfa['Bairro'] = dfa['Bairro'].str.replace(' - Belém (PA)', '', regex=False).str.strip()
dfb['Bairro'] = dfb['Bairro'].str.replace(' - Belém (PA)', '', regex=False).str.strip()
dfc['Bairro'] = dfc['Bairro'].str.replace(' - Belém (PA)', '', regex=False).str.strip()

dfA['Bairro'] = dfA['Bairro'].str.replace('- Belém (PA)', '', regex=False).str.strip()
dfB['Bairro'] = dfB['Bairro'].str.replace('- Belém (PA)', '', regex=False).str.strip()
dfC['Bairro'] = dfC['Bairro'].str.replace('- Belém (PA)', '', regex=False).str.strip()
dfD['Bairro'] = dfD['Bairro'].str.replace('- Belém (PA)', '', regex=False).str.strip()


## 3. verificando se os bairros são os mesmos nas duas tabelas.

In [99]:
#Verificar se são os mesmo bairros
a=dfa['Bairro'].unique().tolist()
A=dfA['Bairro'].unique().tolist()
print(list(set(B) - set(a)))

[]


## 4. Salvando os novos dados

In [102]:
# salvar os novos dados
def salvar_csv(df, caminho):
    if os.path.exists(caminho):
        print(f"⚠️ Arquivo já existe e NÃO será sobrescrito: {caminho}")
    else:
        df.to_csv(caminho, index=False)
        print(f"✅ Arquivo salvo: {caminho}")

path = '/home/akel/PycharmProjects/Data_ambiental/data/process/'

salvar_csv(dfa, path + 'tabela3033a.csv')
salvar_csv(dfb, path + 'tabela3033b.csv')
salvar_csv(dfc, path + 'tabela3033c.csv')

salvar_csv(dfA, path + 'tabela3170A.csv')
salvar_csv(dfB, path + 'tabela3170B.csv')
salvar_csv(dfC, path + 'tabela3170C.csv')
salvar_csv(dfD, path + 'tabela3170D.csv')

⚠️ Arquivo já existe e NÃO será sobrescrito: /home/akel/PycharmProjects/Data_ambiental/data/process/tabela3033a.csv
⚠️ Arquivo já existe e NÃO será sobrescrito: /home/akel/PycharmProjects/Data_ambiental/data/process/tabela3033b.csv
⚠️ Arquivo já existe e NÃO será sobrescrito: /home/akel/PycharmProjects/Data_ambiental/data/process/tabela3033c.csv
⚠️ Arquivo já existe e NÃO será sobrescrito: /home/akel/PycharmProjects/Data_ambiental/data/process/tabela3170A.csv
⚠️ Arquivo já existe e NÃO será sobrescrito: /home/akel/PycharmProjects/Data_ambiental/data/process/tabela3170B.csv
⚠️ Arquivo já existe e NÃO será sobrescrito: /home/akel/PycharmProjects/Data_ambiental/data/process/tabela3170C.csv
⚠️ Arquivo já existe e NÃO será sobrescrito: /home/akel/PycharmProjects/Data_ambiental/data/process/tabela3170D.csv
